In [ ]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from postBornCrossSpectra import PostBorn_Bispec
import numpy as np
import Cosmology as C
import pickle
from scipy.integrate import simps
from scipy.interpolate import RectBivariateSpline
from classy import Class

import copy
import kernels
plt.style.use(['seaborn-colorblind','paper'])
plt.style.use('classic')

In [ ]:
path = '/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/CrossPostBorn/data/class_postlimberdensity/inifiles/'

In [ ]:
zmin=1.0
zmax=1.2
filename = 'class_z%03dz%03d_cl.dat'%(zmin*100,zmax*100)
step = 0.02

In [ ]:
corrs = np.loadtxt(path+filename)
print(corrs.shape)

In [ ]:
zs = np.arange(zmin,zmax+step,step)

In [ ]:
L = corrs[:,0]
print(L)

In [ ]:
print(len(corrs[0,:]))

In [ ]:
ii=1
plt.loglog(corrs[:,0],corrs[:,ii])
plt.plot(abs(corrs[:,0]),(corrs[:,ii+1]))
plt.plot(abs(corrs[:,0]),(corrs[:,ii+2]))
plt.plot((corrs[:,0]),(corrs[:,ii+3]))
plt.semilogy(abs(corrs[:,0]),abs(corrs[:,ii+4]))
plt.semilogy(abs(corrs[:,0]),abs(corrs[:,ii+5]))


In [ ]:
from scipy.interpolate import interp1d
func = interp1d(L[0:250],corrs[0:250,2],bounds_error=False,fill_value=0.)
func2= interp1d(L[0:100],corrs[0:100,3],bounds_error=False,fill_value=0.)

In [ ]:
l=np.linspace(0,300,800)
plt.plot(l,func(l))
plt.plot(l,func2(l))
plt.plot(L[0:300],corrs[0:300,2],ls='--',c='y')
plt.plot(L[0:300],corrs[0:300,3],ls='--',c='m')

In [ ]:
jj = 10

In [ ]:
L[jj]

In [ ]:
indices = np.zeros((66,2))
count = 0
for ii in np.arange(11):
    for nn in np.arange(ii,11):
        indices[count]=[ii,nn]
        count+=1
print(indices)

In [ ]:
zs    = np.arange(0.,5+0.01,step=0.02)
zs[0] = 0.0


zmins = np.arange(0.0,4.9,0.1)
zmins[0] = 0.0
zmaxs = np.arange(0.2,5+0.1,0.1)
print(zmins)
print(zmaxs)

In [ ]:
interp_matrix = np.zeros((len(zs),len(zs)))


In [ ]:
step    = 0.02

def get_matrix_at_L(Lind):
    filenum = 0
    
    zmin    = zmins[filenum]
    zmax    = zmaxs[filenum]
    filename = 'class_z%03dz%03d_cl.dat'%(zmin*100,zmax*100)
    corrs    = np.loadtxt(path+filename)
    corrs    = corrs[:,1::]
    print(corrs.shape)
    for ii in range(0,len(zs)):
        #print('ii',ii)
        for jj in range(ii,ii+1): #track correlation only for delta z = 0.5
            if jj < len(zs):
                z1 = zs[ii]
                z2 = zs[jj]
                #print(z1,z2)
                if z2<zmax+0.001:
                    ##print('%.2f, %.2f, %.2f, %.2f'%(z1,z2,zmin,zmax))
                    filename = 'class_z%03dz%03d_cl.dat'%(zmin*100,zmax*100)
                    corrs    = np.loadtxt(path+filename)
                    corrs    = corrs[:,1::]
                    ind1 = int((z1-zmin)/step)
                    ind2 = int((z2-zmin)/step)
                    index = np.all(np.equal(indices,[ind1,ind2]),axis=1)
                    val = corrs[Lind,index]
                    #print(ind1,ind2,np.where(index))
                interp_matrix[ii,jj] = val

                interp_matrix[jj,ii] = interp_matrix[ii,jj] #hopefully creates only a pointer :)
    return interp_matrix


#interp_matrix = get_matrix_at_L(100)
#print interp_matrix

In [ ]:
data = C.CosmoData(C.Planck2015[1],np.linspace(0.,5.,200))
zs[0]=0.01
chi  = data.chi(zs)

In [ ]:
from scipy.interpolate import RectBivariateSpline as RBS
from scipy.interpolate import interp2d

zs1, zs2 = np.meshgrid(zs,zs)

chi1, chi2 = np.meshgrid(chi,chi)

func = RBS(chi,chi,interp_matrix,kx=1,ky=1)#, bounds_error=False, fill_value=0.) #RBS and interp2d are the same, just differnt wrappers

zs_t = np.arange(0,5+0.01,step=0.01)

chi_ = data.chi(zs_t)

spline = func(chi,chi)

In [ ]:
plt.figure()
plt.imshow((interp_matrix)[0:15,0:15],vmin=-20,vmax=1)
#plt.imshow(np.log(-interp_matrix),vmin=-20,vmax=1)
plt.colorbar()
plt.show()
print(interp_matrix[0:15,0:15])

In [ ]:
plt.figure()
plt.imshow(np.log(abs(spline)),vmin=-20,vmax=1)
plt.imshow(np.log(-spline),vmin=-20,vmax=1)
plt.colorbar()
plt.show()


In [ ]:
plt.plot(interp_matrix[0,0:20],marker='o')

plt.plot(interp_matrix[100,100:120],marker='o')

#plt.plot(abs(spline[0,0:20]),marker='^')
# plt.plot(spline[1,1:42],marker='^')
# plt.plot(spline[100,100:240],marker='^')

print(spline[0,0:20])

In [ ]:
plt.plot(interp_matrix[1,1:21],marker='o')

In [ ]:
plt.plot(interp_matrix[200,200:220],marker='o')

In [ ]:
## Appending the first file with zeros to make all files the same size

# path
# fname = 'class_z000z020_cl.dat'
# cl = np.loadtxt(path + fname)
# fname = 'class_z010z030_cl.dat'
# clall = np.loadtxt(path + fname)
# clnew = np.zeros_like(clall)


# for i in range(cl.shape[1]):
#     clnew[:cl.shape[0], i] = cl[:, i]
    
# path
# fname = 'class_z000z020_cl.dat'
# # np.savetxt(path + fname, clnew)
# clcheck = np.loadtxt(path + fname)
# clcheck.shape

In [ ]:
from BispectraMain import dNdz_LSST, simple_bias, gal_clus 

def simple_kernel(x,xmax):
    return (xmax-x)/x/xmax

In [ ]:
LSST_bin = 'all'

In [ ]:
W_g = gal_clus(b=simple_bias,bin_num=LSST_bin,cosmo=data, dNdz=dNdz_LSST)

In [ ]:
W_l = simple_kernel

In [ ]:
def Cl_psidelta(chi,z,L,cls,cosmo=data):
    return -cosmo.lens_prefac*(chi/L)**2*(1.+z)*cls/(L*(L+1.)/2./np.pi)

# lens_prefac is 1.5*self.H_0**2.*self.Omega_m0/(const.LIGHT_SPEED**2)

In [ ]:
L_index = np.arange(0,100)
print(L_index)

In [ ]:
cls = []
for L_ind in [0,10,50,100,500,1000]:
    print(L_ind)
    matrix          = get_matrix_at_L(L_ind)
    
    weighted_matrix = W_l(chi1,data.chi_cmb)*W_g(chi2,zs2)*Cl_psidelta(chi1,zs1,L[L_ind],matrix,cosmo=data)
    
    res=[]
    for ii in range(len(chi)):
        res+=[np.trapz(weighted_matrix[ii],chi)]
    
    cls+=[np.trapz(res,chi)]
    
cls = np.asarray(cls)

In [ ]:
L_sample = [L[ii] for ii in [0,10,50,100,500,1000]]
L_sample = np.asarray(L_sample)
print(cls.shape)

In [ ]:
LL, ll, theta = pickle.load(open('/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/ells/ell_ang_full_Lmin1_Lmax3000_lmin1_lmax8000_lenL120_lenl140_lenang120_1e-04.pkl','r'))
CLkg = np.load('/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/power_spectra/CL_kg_LSSTbinall_full_Planck2015_Lmin1-Lmax2999_.npy')
CLgg = np.load('/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/power_spectra/CL_gg_LSSTbinall_full_Planck2015_Lmin1-Lmax2999_.npy')
CLkk = np.load('/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/outputs/power_spectra/CL_kk_LSSTbinall_full_Planck2015_Lmin1-Lmax2999_.npy')

In [ ]:
plt.figure()
plt.loglog(L_sample, -L_sample**2*cls, marker ='o', ls='')
plt.semilogx(LL,CLkg)

In [ ]:
plt.semilogx(L_sample, -L_sample**2*cls/np.interp(L_sample,LL,CLkg), marker ='o', ls='')